In [1]:
import re
import kss
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
f = open('text/CM.txt', encoding="utf8")
i=0
gogo= 0
sentence_list = []
while True:
    line = f.readline()
    if not line: break
    if "<!DOCTYPE" in line:
        gogo = 0
    if "<text>" in line:
        gogo = 1
    if "</text>" in line:
        gogo = 0
    if gogo == 0: continue
    if line != '\n':
        i=i+1
        line = re.sub('<trunc>.+?<\/trunc>', '', line, 0, re.S)
        line = re.sub('<.+?>', '', line, 0, re.S)
        line = re.sub('\(.+?\)', '', line, 0, re.S)
        line = re.sub('\[.+?\]', '', line, 0, re.S)
        line = re.sub('\{.+?\}', '', line, 0, re.S)
        line = re.sub('《.+?》', '', line, 0, re.S)
        line = re.sub('[-=+,#/\?:^$@*\"※~&%ㆍ!『』\‘|\(\)\[\]\<\>`\'…《》\{\}_「」±√]', '', line,0,re.S)
        line = line.lower()
        if len(line) >4 and len(line)<10:
            sentence_list+= kss.split_sentences(line)
f.close()
print(len(sentence_list))

51916


In [2]:
#print(sentence_list[:5])
choseong_list = [char for char in "ㄱㄲㄴㄷㄸㄹㅁㅂㅃㅅㅆㅇㅈㅉㅊㅋㅌㅍㅎ"]
word_list = []
target = []
for sentence in sentence_list:
    sen2word = ""
    for word in sentence:
        if re.match('[가-힣]',word) is not None :
            sen2word+=choseong_list[(int)((ord(word)-ord('가'))/(21*28))]
        else :
            sen2word+=word
    target.append('\t'+sentence+'\n')
    word_list.append(sen2word)
    
#print(target[:5])
#print(word_list[:5])
src_vocab=set()
for words in word_list: # 1줄씩 읽음
    for char in words: # 1개의 글자씩 읽음
        src_vocab.add(char)

tar_vocab=set()
for line in target:
    for char in line:
        tar_vocab.add(char)
        
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
#print(src_vocab_size)
#print(tar_vocab_size)
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
#print(src_vocab)
#print(tar_vocab[10:20])
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
#print(src_to_index)
#print(tar_to_index)
encoder_input = []
for line in word_list: #입력 데이터에서 1줄씩 문장을 읽음
    temp_X = []
    for w in line: #각 줄에서 1개씩 글자를 읽음
      temp_X.append(src_to_index[w]) # 글자를 해당되는 정수로 변환
    encoder_input.append(temp_X)
#print(encoder_input[:5])
decoder_input = []
for line in target:
    temp_X = []
    for w in line:
      temp_X.append(tar_to_index[w])
    decoder_input.append(temp_X)
#print(decoder_input[:5])
decoder_target = []
for line in target:
    t=0
    temp_X = []
    for w in line:
      if t>0:
        temp_X.append(tar_to_index[w])
      t=t+1
    decoder_target.append(temp_X)
#print(decoder_target[:5])
max_src_len = max([len(line) for line in word_list])
max_tar_len = max([len(line) for line in target])
#print(max_src_len)
print(max_tar_len)
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

encoder_inputer = encoder_input
decoder_inputer = decoder_input
decoder_targeter = decoder_target

encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# encoder_outputs도 같이 리턴받기는 했지만 여기서는 필요없으므로 이 값은 버림.
encoder_states = [state_h, state_c]
# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 바로 은닉 상태와 셀 상태.

decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 디코더의 첫 상태를 인코더의 은닉 상태, 셀 상태로 합니다.
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
    
for timer in range(5):
    encoder_inputer = encoder_inputer[timer*1000:(timer+1)*10000]
    decoder_inputer = decoder_inputer[timer*1000:(timer+1)*10000]
    decoder_targeter = decoder_targeter[timer*1000:(timer+1)*10000]

    encoder_input = to_categorical(encoder_inputer, num_classes=src_vocab_size)
    decoder_input = to_categorical(decoder_inputer, num_classes=tar_vocab_size)
    decoder_target = to_categorical(decoder_targeter, num_classes=tar_vocab_size)
    
    model.fit(x=[encoder_input, decoder_input], y=decoder_target, epochs=200)
    #model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=200)

10
Train on 10000 samples
Epoch 1/200
10000/10000 [==============================] - 43s 4ms/sample - loss: 2.9875
Epoch 2/200
10000/10000 [==============================] - 39s 4ms/sample - loss: 2.4117
Epoch 3/200
10000/10000 [==============================] - 39s 4ms/sample - loss: 2.0726
Epoch 4/200
10000/10000 [==============================] - 39s 4ms/sample - loss: 1.8481
Epoch 5/200
10000/10000 [==============================] - 39s 4ms/sample - loss: 1.6564
Epoch 6/200
10000/10000 [==============================] - 40s 4ms/sample - loss: 1.4968
Epoch 7/200
10000/10000 [==============================] - 39s 4ms/sample - loss: 1.3661
Epoch 8/200
10000/10000 [==============================] - 40s 4ms/sample - loss: 1.2592
Epoch 9/200
10000/10000 [==============================] - 39s 4ms/sample - loss: 1.1690
Epoch 10/200
10000/10000 [==============================] - 40s 4ms/sample - loss: 1.0951
Epoch 11/200
10000/10000 [==============================] - 39s 4ms/sample - loss: 

KeyboardInterrupt: 

In [ ]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [ ]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
decoder_states = [state_h, state_c]
# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [ ]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [ ]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)
    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition: # stop_condition이 True가 될 때까지 루프 반복
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_tar_len):
            stop_condition = True

        # 길이가 1인 타겟 시퀀스를 업데이트 합니다.
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 상태를 업데이트 합니다.
        states_value = [h, c]

    return decoded_sentence

In [ ]:
import numpy as np
i = 0
for seq_index in [105,7000,15502,40015,5]: # 입력 문장의 인덱스
    i+=1
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', word_list[seq_index])
    print('정답 문장:', target[seq_index][1:len(target[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력